In [2]:
import os
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo
from pymongo import MongoClient
print ('Mongo version', pymongo.__version__)
client = MongoClient('localhost', 27017)
db = client.test
collection = db.people

Mongo version 3.7.2


In [6]:
collection.drop()
os.system('mongoimport -d test -c people dummyData.json')

0

In [7]:
cursor = collection.find().sort('Age',pymongo.ASCENDING).limit(3)
for doc in cursor:
    print(doc)

{'_id': ObjectId('5be4464d8d931c7e82c6a74e'), 'Name': 'Sawyer, Neve M.', 'Age': 18, 'Country': 'Serbia', 'Location': '-34.37446, 174.0838'}
{'_id': ObjectId('5be4464d8d931c7e82c6a719'), 'Name': 'Townsend, Cadman I.', 'Age': 19, 'Country': 'Somalia', 'Location': '-87.69188, -144.16138'}
{'_id': ObjectId('5be4464d8d931c7e82c6a734'), 'Name': 'Graham, Emerald O.', 'Age': 20, 'Country': 'Eritrea', 'Location': '61.35398, 28.04381'}


In [17]:
pipeline = [
    {"$group":{"_id":"$Country",
        "AvgAge":{"$avg":"$Age"},
               "Count":{"$sum":1},
    }
    },
    {"$sort":{"Count":-1,"AvaAge":1}}
]
aggResult = collection.aggregate(pipeline) # return a cursor
df1 = pd.DataFrame(list(aggResult)) # use list to turn the cursor to an array of documents
df1 = df1.set_index("_id") # set index use _id
df1.head() # return fist n rows, by default n=5

,AvgAge,Count
_id,,
China,46.250000,4
Antarctica,46.333333,3
Guernsey,48.333333,3
"Saint Helena, Ascension and Tristan da Cunha",35.000000,2
Mauritania,53.500000,2


In [22]:
pipeline = [
    {"$match":{"Country":"China"}},
]
aggResult = collection.aggregate(pipeline)
df2 = pd.DataFrame(list(aggResult))
df2.head()

,Age,Country,Location,Name,_id
0,32,China,"39.9127, 116.3833","Holman, Hasad O.",5be4464d8d931c7e82c6a708
1,43,China,"31.2, 121.5","Byrd, Dante A.",5be4464d8d931c7e82c6a747
2,57,China,"45.75, 126.6333","Carney, Tamekah I.",5be4464d8d931c7e82c6a74f
3,53,China,"40, 95","Mayer, Violet U.",5be4464d8d931c7e82c6a75b


In [38]:
import folium
print ('Folium version', folium.__version__)

world_map = folium.Map(location=[35,100],
                      zoom_start=4)
#world_map.simple_marker()
for i in range(len(df2)):
    loc = df2.Location[i].split(',')
    folium.Marker(location=[float(loc[0]),float(loc[1])], popup=df2.Name[i]+', age:'+str(df2.Age[i])).add_to(world_map)
    
world_map

Folium version 0.6.0
